In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
import xgboost as xgb
import optuna
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, make_scorer

#### Objective
House price prediction

In [3]:
dataset = fetch_california_housing()

In [4]:
df = pd.DataFrame(dataset.data)
df.columns = dataset.feature_names
df[dataset.target_names[0]] = dataset.target


In [5]:

df = df.dropna()

In [6]:
X, y = df.iloc[:,:-1], df.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

In [7]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
def objective_linear(trial):
    params = {
        'degree': trial.suggest_int('degree', 1, 5)
    }
    model = Pipeline([
        ('poly', PolynomialFeatures(**params)),
        ('linear', LinearRegression(fit_intercept=False))
    ])
    score = cross_val_score(model, X_train, y_train, cv=kf, scoring=make_scorer(mean_squared_error)).mean()
    return score

def objective_xg(trial):
    params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'booster': trial.suggest_categorical('booster', ['gbtree', 'gblinear', 'dart']),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        # 'max_depth': trial.suggest_int('max_depth', 3, 10),
        # 'subsample': trial.suggest_float('subsample', 0.5, 1),
        # 'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1),
        'lambda': trial.suggest_float('lambda', 1e-5, 1),
        'alpha': trial.suggest_float('alpha', 1e-5, 1),
        # 'min_child_weight': trial.suggest_int('min_child_weight', 1, 10)
    } # some hyperparameters removed due to error messages at runtime
    model = xgb.XGBRegressor(**params)
    score = cross_val_score(model, X_train, y_train, cv=kf, scoring=make_scorer(mean_squared_error)).mean()
    return score

def objective_svr(trial):
    params = {
        'kernel': trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly']),
        'C': trial.suggest_float('C', 0.1, 10),
        'epsilon': trial.suggest_float('epsilon', 0.01, 0.1),
    }
    model = SVR(**params)
    score = cross_val_score(model, X_train, y_train, cv=kf, scoring=make_scorer(mean_squared_error)).mean()
    return score

def objective_decision_reg(trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', 2, 10),
        'min_samples_split': trial.suggest_float('min_samples_split', 0.1, 1),
        'min_samples_leaf': trial.suggest_float('min_samples_leaf', 0.1, 0.5),
        'max_features': trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2', None])
    }
    model = DecisionTreeRegressor(**params)
    score = cross_val_score(model, X_train, y_train, cv=kf, scoring=make_scorer(mean_squared_error)).mean()
    return score

def objective_rf_reg(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 100),
        'max_depth': trial.suggest_int('max_depth', 2, 32, log=True),
        'min_samples_split': trial.suggest_float('min_samples_split', 0.1, 1),
        'min_samples_leaf': trial.suggest_float('min_samples_leaf', 0.1, 0.5),
        'max_features': trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2', None])
    }
    model = RandomForestRegressor(**params)
    score = cross_val_score(model, X_train, y_train, cv=kf, scoring=make_scorer(mean_squared_error)).mean()
    return score

In [21]:
err_rates = []

objective_funcs = [objective_linear, objective_xg, objective_decision_reg, objective_rf_reg]
# SVR removed due to slow training - would run on GPU if possible
# objective_funcs = [objective_linear, objective_xg, objective_svr, objective_decision_reg, objective_rf_reg]
for func in objective_funcs:
    study = optuna.create_study(direction='minimize')
    study.optimize(func, n_trials=20)
    err_rates.append({'Model': func.__name__, 'Best error': study.best_value, 'Best params': study.best_params})

[I 2023-12-30 16:07:21,586] A new study created in memory with name: no-name-747fa7f0-a30b-4d14-a615-29df063ed46f
[I 2023-12-30 16:07:21,620] Trial 0 finished with value: 0.5193270208432356 and parameters: {'degree': 1}. Best is trial 0 with value: 0.5193270208432356.
[I 2023-12-30 16:07:23,846] Trial 1 finished with value: 7697552590.333864 and parameters: {'degree': 4}. Best is trial 0 with value: 0.5193270208432356.
[I 2023-12-30 16:07:23,886] Trial 2 finished with value: 0.5193270208432356 and parameters: {'degree': 1}. Best is trial 0 with value: 0.5193270208432356.
[I 2023-12-30 16:07:24,618] Trial 3 finished with value: 542180.5337066596 and parameters: {'degree': 3}. Best is trial 0 with value: 0.5193270208432356.
[I 2023-12-30 16:07:24,663] Trial 4 finished with value: 0.5193270208432356 and parameters: {'degree': 1}. Best is trial 0 with value: 0.5193270208432356.
[I 2023-12-30 16:07:36,177] Trial 5 finished with value: 80447209847.1964 and parameters: {'degree': 5}. Best is 

In [22]:
df_err = pd.DataFrame(err_rates)
df_err

,Model,Best error,Best params
0,objective_linear,0.519327,{'degree': 1}
1,objective_xg,0.220817,"{'booster': 'gbtree', 'learning_rate': 0.18113..."
2,objective_decision_reg,0.658636,"{'max_depth': 7, 'min_samples_split': 0.114840..."
3,objective_rf_reg,0.789714,"{'n_estimators': 25, 'max_depth': 16, 'min_sam..."


In [23]:
chosen_model = df_err.iloc[df_err['Best error'].idxmin()]
chosen_model

Model                                               objective_xg
Best error                                              0.220817
Best params    {'booster': 'gbtree', 'learning_rate': 0.18113...
Name: 1, dtype: object

In [24]:
model = xgb.XGBRegressor(**chosen_model['Best params'])
model.fit(X_train, y_train)
model.save_model('0001.json')
y_pred_test = model.predict(X_test)
rmse_test = mean_squared_error(y_test, y_pred_test)
rmse_test

0.2156787750266574

In [25]:
df_results = X_test.copy()
df_results['MedVal'] = y_test
df_results['PredVal'] = y_pred_test
df_results

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedVal,PredVal
20046,1.6812,25.0,4.192201,1.022284,1392.0,3.877437,36.06,-119.01,0.47700,0.568279
3024,2.5313,30.0,5.039384,1.193493,1565.0,2.679795,35.14,-119.46,0.45800,0.913133
15663,3.4801,52.0,3.977155,1.185877,1310.0,1.360332,37.80,-122.44,5.00001,5.338040
20484,5.7376,17.0,6.163636,1.020202,1705.0,3.444444,34.28,-118.72,2.18600,2.504100
9814,3.7250,34.0,5.492991,1.028037,1063.0,2.483645,36.62,-121.93,2.78000,2.445643
...,...,...,...,...,...,...,...,...,...,...
15362,4.6050,16.0,7.002212,1.066372,1351.0,2.988938,33.36,-117.22,2.63300,2.464914
16623,2.7266,28.0,6.131915,1.256738,1650.0,2.340426,35.36,-120.83,2.66800,2.028810
18086,9.2298,25.0,7.237676,0.947183,1585.0,2.790493,37.31,-122.05,5.00001,5.112632
2144,2.7850,36.0,5.289030,0.983122,1227.0,2.588608,36.77,-119.76,0.72300,0.691650


In [26]:
new_data = pd.DataFrame({
    'MedInc': [8.7038],
    'HouseAge': [10.0],
    'AveRooms': [5.037594],
    'AveBedrms': [5.982048],
    'Population': [1000.0],
    'AveOccup': [4.04739],
    'Latitude': [36.0],
    'Longitude': [-120.04]
})

prediction = model.predict(new_data)
prediction

array([3.9936025], dtype=float32)